In [3]:
import numpy as np
import matplotlib.pyplot as plt

def normalizar(vetor):
    return vetor / np.linalg.norm(vetor)

def refletir(vetor, axis):
    return vetor - 2 * np.dot(vetor, axis) * axis

def intersecao_esfera(centro, raio, origem_raio, direcao_raio):
    b = 2 * np.dot(direcao_raio, origem_raio - centro)
    c = np.linalg.norm(origem_raio - centro) ** 2 - raio ** 2
    delta = b ** 2 - 4 * c
    if delta > 0:
        t1 = (-b + np.sqrt(delta)) / 2
        t2 = (-b - np.sqrt(delta)) / 2
        if t1 > 0 and t2 > 0:
            return min(t1, t2)
    return None

def intersecao_objeto_mais_perto(objects, origem_raio, direcao_raio):
    distancias = [intersecao_esfera(obj['centro'], obj['raio'], origem_raio, direcao_raio) for obj in objects]
    objeto_mais_proximo = None
    distancia_minima = np.inf
    for index, distancia in enumerate(distancias):
        if distancia and distancia < distancia_minima:
            distancia_minima = distancia
            objeto_mais_proximo = objects[index]
    return objeto_mais_proximo, distancia_minima

largura = 300
altura = 200

profundidade_maxima = 3

camera = np.array([0, 0, 1])
aspecto = float(largura) / altura
tela = (-1, 1 / aspecto, 1, -1 / aspecto)

luz = { 'posicao': np.array([5, 5, 5]), 
       'ambiente': np.array([1, 1, 1]), 
       'difuso': np.array([1, 1, 1]), 
       'especular': np.array([1, 1, 1]) 
       }

# objects = [
#     { 'centro': np.array([-0.2, 0, -1]), 
#      'raio': 0.5, 
#      'ambiente': np.array([0.1, 0, 0]), 
#      'difuso': np.array([0.7, 0, 0]), 
#      'especular': np.array([1, 1, 1]), 
#      'brilho': 100, 
#      'reflexao': 0.5 
#      },
#     { 'centro': np.array([-0.3, 0, 0]), 
#      'raio': 0.2, 
#      'ambiente': np.array([0, 0.1, 0]), 
#      'difuso': np.array([0, 0.6, 0]), 
#      'especular': np.array([1, 1, 1]), 
#      'brilho': 100, 
#      'reflexao': 0.5 
#      },
#     { 'centro': np.array([0, -9000, 0]), 
#      'raio': 9000 - 0.7, 
#      'ambiente': np.array([0.1, 0.1, 0.1]),
#      'difuso': np.array([0.6, 0.6, 0.6]), 
#      'especular': np.array([1, 1, 1]),
#      'brilho': 100, 
#      'reflexao': 0.5 
#      }
# ]

# objects = [
#     { 'centro': np.array([-0.2, 0, -1]), 
#      'raio': 0.7, 
#      'ambiente': np.array([0.1, 0, 0]), 
#      'difuso': np.array([0.7, 0, 0]), 
#      'especular': np.array([1, 1, 1]),
#      'brilho': 100, 
#      'reflexao': 0.5 
#      },
#     { 'centro': np.array([0.1, -0.3, 0]), 
#      'raio': 0.1, 
#      'ambiente': np.array([0.1, 0, 0.1]),
#      'difuso': np.array([0.7, 0, 0.7]), 
#      'especular': np.array([1, 1, 1]), 
#      'brilho': 100,
#      'reflexao': 0.5
#      },
#     { 'centro': np.array([-0.3, 0, 0]), 
#      'raio': 0.15, 
#      'ambiente': np.array([0, 0.1, 0]), 
#      'difuso': np.array([0, 0.6, 0]), 
#      'especular': np.array([1, 1, 1]),
#      'brilho': 100, 
#      'reflexao': 0.5 
#      },
#     { 'centro': np.array([0, -9000, 0]), 
#      'raio': 9000 - 0.7,
#      'ambiente': np.array([0.1, 0.1, 0.1]), 
#      'difuso': np.array([0.6, 0.6, 0.6]),
#      'especular': np.array([1, 1, 1]), 
#      'brilho': 100, 
#      'reflexao': 0.5 
#      }
# ]

objects = [
    { 'centro': np.array([-0.2, 0, -1]), 
     'raio': 0.7, 
     'ambiente': np.array([0.1, 0, 0]), 
     'difuso': np.array([0.7, 0, 0]), 
     'especular': np.array([1, 1, 1]),
     'brilho': 100, 
     'reflexao': 0.5 
     }
]

imagem = np.zeros((altura, largura, 3))

for i, y in enumerate(np.linspace(tela[1], tela[3], altura)):
    for j, x in enumerate(np.linspace(tela[0], tela[2], largura)):
       
        # tela origem
        pixel = np.array([x, y, 0])
        origem = camera
        direcao = normalizar(pixel - origem)
        cor = np.zeros((3))
        reflexao = 1

        for k in range(profundidade_maxima):

            # checar intersecao
            objeto_mais_proximo, distancia_minima = intersecao_objeto_mais_perto(objects, origem, direcao)
            if objeto_mais_proximo is None:
                break

            intersecao = origem + distancia_minima * direcao
            normalizar_superficie = normalizar(intersecao - objeto_mais_proximo['centro'])
            ponto_troca = intersecao + 1e-5 * normalizar_superficie
            intersecao_da_luz = normalizar(luz['posicao'] - ponto_troca)

            _, distancia_minima = intersecao_objeto_mais_perto(objects, ponto_troca, intersecao_da_luz)
            intersecao_da_luz_distancia = np.linalg.norm(luz['posicao'] - intersecao)
            tem_sombra = distancia_minima < intersecao_da_luz_distancia

            if tem_sombra:
                break

            iluminacao = np.zeros((3))

            # ambiente
            iluminacao += objeto_mais_proximo['ambiente'] * luz['ambiente']

            # difuso
            iluminacao += objeto_mais_proximo['difuso'] * luz['difuso'] * np.dot(intersecao_da_luz, normalizar_superficie)

            # especular
            intersecao_camera = normalizar(camera - intersecao)
            H = normalizar(intersecao_da_luz + intersecao_camera)
            iluminacao += objeto_mais_proximo['especular'] * luz['especular'] * np.dot(normalizar_superficie, H) ** (objeto_mais_proximo['brilho'] / 4)

            # reflexao
            cor += reflexao * iluminacao
            reflexao *= objeto_mais_proximo['reflexao']

            origem = ponto_troca
            direcao = refletir(direcao, normalizar_superficie)

        imagem[i, j] = np.clip(cor, 0, 1)
    print("%d/%d" % (i + 1, altura))

plt.imsave('imagem.png', imagem)

1/200
2/200
3/200
4/200
5/200
6/200
7/200
8/200
9/200
10/200
11/200
12/200
13/200
14/200
15/200
16/200
17/200
18/200
19/200
20/200
21/200
22/200
23/200
24/200
25/200
26/200
27/200
28/200
29/200
30/200
31/200
32/200
33/200
34/200
35/200
36/200
37/200
38/200
39/200
40/200
41/200
42/200
43/200
44/200
45/200
46/200
47/200
48/200
49/200
50/200
51/200
52/200
53/200
54/200
55/200
56/200
57/200
58/200
59/200
60/200
61/200
62/200
63/200
64/200
65/200
66/200
67/200
68/200
69/200
70/200
71/200
72/200
73/200
74/200
75/200
76/200
77/200
78/200
79/200
80/200
81/200
82/200
83/200
84/200
85/200
86/200
87/200
88/200
89/200
90/200
91/200
92/200
93/200
94/200
95/200
96/200
97/200
98/200
99/200
100/200
101/200
102/200
103/200
104/200
105/200
106/200
107/200
108/200
109/200
110/200
111/200
112/200
113/200
114/200
115/200
116/200
117/200
118/200
119/200
120/200
121/200
122/200
123/200
124/200
125/200
126/200
127/200
128/200
129/200
130/200
131/200
132/200
133/200
134/200
135/200
136/200
137/200
138/200
139/